In [1]:
#!g1.1
import os, sys
import re, random
import copy
import json
import pympi
import pymystem3
import pandas as pd
import numpy as np
from pprint import pprint
from itertools import chain
from tqdm import tqdm

sys.path.append('../')
from pyeaf.pyeaf import EAFReader
from pyeaf.text import VocabularyVectorizer, TextStemmer, RSLStemmer, GramBinarizer
from model.model import device

* существительные в именительном падеже
* существительные в винительном падеже
* переходные глаголы

In [2]:
#!g1.1
er = EAFReader(directory="../Разметки eaf")
er.load()

In [3]:
#!g1.1
sentences_rus = er.get_sentences(er.RUS)
sentences_rsl = er.get_sentences(er.RSL_R)

# sentences_rsl_left = er.get_sentences(er.RSL_L)

In [4]:
#!g1.1
st = TextStemmer()

In [5]:
#!g1.1

stem_sentences_rus, gram_sentences_rus = st.stem(sentences_rus, gram=True)
stem_sentences_rsl = list(map(RSLStemmer.stem_sentence, sentences_rsl))

Prep for generation

In [6]:
#!g1.1
with open("animate.txt", "r", encoding="utf-8") as f:
    anim_nouns = [line.strip('\n') for line in f.readlines()]

In [7]:
#!g1.1
with open("inanimate.txt", "r", encoding="utf-8") as f:
    inanim_nouns = [line.strip('\n') for line in f.readlines()]

In [8]:
#!g1.1
def split_by_gender(nouns):
    nouns_m, nouns_f, nouns_n = [], [], []
    for noun in nouns:
        if 'муж' in st.stem(noun)[1][0][0]:
            nouns_m.append(noun)
        elif 'жен' in st.stem(noun)[1][0][0]:
            nouns_f.append(noun)
        elif 'сред' in st.stem(noun)[1][0][0]:
            nouns_n.append(noun)
    return nouns_m, nouns_f, nouns_n

In [9]:
#!g1.1
inan_m_nouns, inan_f_nouns, inan_n_nouns = split_by_gender(inanim_nouns)
anim_m_nouns, anim_f_nouns, anim_n_nouns = split_by_gender(anim_nouns)

In [10]:
#!g1.1
from string import punctuation

noun_np_gen = []
for line in open("patterns.noun_np_gen.txt", "r", encoding="utf-8"):
    if not (set(line) & set(punctuation)):
        noun_np_gen.append(line.strip('\n'))

In [11]:
#!g1.1
# разделить на одуш\неодуш
anim_noun_np_gen = []
inanim_noun_np_gen = []
for np in noun_np_gen:
    main_word = np.split()[0]
    if 'неод' in st.stem(main_word)[1][0][0]:
        inanim_noun_np_gen.append(np)
    else:
        anim_noun_np_gen.append(np)

In [12]:
#!g1.1
inanim_m_noun_np_gen, inanim_f_noun_np_gen, inanim_n_noun_np_gen = [], [], []
for np in inanim_noun_np_gen:
    noun = np.split()[0]
    if 'муж' in st.stem(noun)[1][0][0]:
        inanim_m_noun_np_gen.append(np)
    elif 'жен' in st.stem(noun)[1][0][0]:
        inanim_f_noun_np_gen.append(np)
    elif 'сред' in st.stem(noun)[1][0][0]:
        inanim_n_noun_np_gen.append(np)

In [13]:
#!g1.1
anim_m_noun_np_gen, anim_f_noun_np_gen, anim_n_noun_np_gen = [], [], []
for np in anim_noun_np_gen:
    noun = np.split()[0]
    if 'муж' in st.stem(noun)[1][0][0]:
        anim_m_noun_np_gen.append(np)
    elif 'жен' in st.stem(noun)[1][0][0]:
        anim_f_noun_np_gen.append(np)
    elif 'сред' in st.stem(noun)[1][0][0]:
        anim_n_noun_np_gen.append(np)

In [14]:
#!g1.1
words = pd.read_csv("animation_pos.csv", encoding="utf-8")

In [15]:
#!g1.1
tr_verbs, intr_verbs = [], []
for verb in words[words['pos'] == 'V'].itertuples():
    analysis = json.loads(verb.analysis.replace('\'', '\"'))
    gr = analysis[0]['analysis'][0]['gr']
    if 'пе' in gr:
        tr_verbs.append(verb.text)
    elif 'нп' in gr:
        intr_verbs.append(verb.text)

## Generation itself

should turn it into class...

In [6]:
#!g1.1
depth = lambda L: isinstance(L, list) and max(map(depth, L))+1

def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

def drop_inner_nest(S):
    for sublist in S:
        if isinstance(sublist[0], list):
            yield sublist[0]
        else:
            yield sublist

def create_gr_n_tags(gr, old_gram_sent_rus):
    # keep original case and number
    tags = ['им', 'вин', 'род', 'дат', 'твор', 'пр', 'парт', 'местн', 'зват', 'ед', 'мн']
    initial_tags = [el for el in old_gram_sent_rus if el in tags]
    for i, word_gr in enumerate(gr):
        gr[i] = [el for el in word_gr if el not in tags]
        gr[i].extend(initial_tags)
    return gr

def create_rsl_gloss(old_rus_lemma, new_rus_lemma, old_stem_sent_rus: list, old_stem_rsl: list):
    """creates rsl glosses for the whole stemmed rsl sentence"""
    # update stemmed rsl sentence
    # return full rsl sentence as a list, not just updated words
    # this does not work well because of things like посмотреть\смотреть
    new_stem_rsl = copy.deepcopy(old_stem_rsl)
    #print('create_rsl_gloss')
    #print('new_rus_lemma', new_rus_lemma)
    found = False
    for i, gloss in enumerate(old_stem_rsl):
        if gloss == old_rus_lemma:
            #print('rus lemma based change')
            new_stem_rsl[i] = new_rus_lemma
            found = True
    if not found:
        # so we have to use index replacement as well
        for j, gloss in enumerate(old_stem_sent_rus):
            if gloss == old_rus_lemma and j < len(new_stem_rsl):
                #print('index based change')
                new_stem_rsl[j] = new_rus_lemma
    #print("new_stem_rsl on this step")
    #pprint(new_stem_rsl)
    return new_stem_rsl

def make_replacments(replacement, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus: list, old_stem_rsl: list, keep_gr=2):
    old_rus_lemma = old_stem_sent_rus_one
    old_stem_sent_rus_one, gr = st.stem(replacement)
    new_rus_lemma = old_stem_sent_rus_one[0]
    if keep_gr==1:
        old_gram_sent_rus_one = create_gr_n_tags(gr, old_gram_sent_rus_one)
    elif keep_gr==0:
        old_gram_sent_rus_one = gr
    new_stem_rsl_one = create_rsl_gloss(old_rus_lemma, new_rus_lemma, old_stem_sent_rus, old_stem_rsl)
    return old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl_one

def pick_and_replace(old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, choices, complex_choices=None):
    if complex_choices:
        # decide between single noun of np with gen
        pick = random.randint(0,1)  # 0 is single noun, 1 is np with gen
        if not bool(pick):
            choice = random.choice(choices)
            if len(choice.split()) > 1:
                old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=1)
                # this line is in the wrong place
            else:
                old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=2)
        else:
            choice = random.choice(complex_choices)
            old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=0)
    else: 
        choice = random.choice(choices)
        old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=0)
    return old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl

def generate_parallel_sentence(old_gram_sent_rus_init, old_stem_sent_rus_init, old_stem_rsl_init):
    """this function generates one parallel sentence"""
    old_gram_sent_rus = copy.deepcopy(old_gram_sent_rus_init)
    old_stem_sent_rus = copy.deepcopy(old_stem_sent_rus_init)
    old_stem_rsl = copy.deepcopy(old_stem_rsl_init)
    number_replacements = 0   
    for k, gram in enumerate(old_gram_sent_rus_init):
        # gender is relevant for nouns because of concord 
        if 'S' and 'од' and 'муж' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, anim_m_nouns, complex_choices=anim_m_noun_np_gen)
        elif 'S' and 'неод'  and 'муж' in gram:
            number_replacements += 1 
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, inan_m_nouns, complex_choices=inanim_m_noun_np_gen)
        elif 'S' and 'од' and 'жен' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, anim_f_nouns, complex_choices=anim_f_noun_np_gen)
        elif 'S' and 'неод'  and 'жен' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, inan_f_nouns, complex_choices=inanim_f_noun_np_gen)
        elif 'S' and 'од' and 'сред' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, anim_n_nouns, complex_choices=anim_n_noun_np_gen)
        elif 'S' and 'неод'  and 'сред' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, inan_n_nouns, complex_choices=inanim_n_noun_np_gen)
        elif 'V' and 'пе' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, tr_verbs)
        elif 'V' and 'нп' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, intr_verbs)

        # если делать больше 5 замен, получается полный бред
        # надо еще сделать замены не подряд, а типа рандомно пропускать шаги??
        # может после переходного глагола всегда пропускать,
        # потому что очень сложно случайным образом подобрать правильный прямой объект или и того хуже инфинитивный оборот
        if number_replacements == 5:
            break
            
    if number_replacements == 0:
        # havent found anything to replace
        return None, None, None
    
    # randomly pick word
    # insert constituent in old_stem_sent_rus (stem\gram before insertion)
    # insert gram of constituent in old_gram_sent_rus it should inherent everything from the original word
    # but if the length of replacement > 1, 
    # insert stemmed constituent in old_stem_rsl
    # flatten lists of depth 3
    if depth(old_gram_sent_rus) > 2:
        old_gram_sent_rus = list(drop_inner_nest(list(drop_inner_nest(old_gram_sent_rus))))

    return old_gram_sent_rus, list(flatten(old_stem_sent_rus)), list(flatten(old_stem_rsl))

In [17]:
#!g1.1
from sklearn.model_selection import train_test_split

stem_sents_rus, test_stem_rus, gram_sents_rus, test_gram_rus, stem_sents_rsl, test_rsl = train_test_split(stem_sentences_rus,
                                                                                                          gram_sentences_rus,
                                                                                                          stem_sentences_rsl,
                                                                                                          test_size=0.2)

In [18]:
#!g1.1
from tqdm import tqdm

new_gram_sents_rus, new_stem_sents_rus, new_stem_rsls = [], [], []

for n, gram in enumerate(tqdm(gram_sents_rus)):
    k = random.randint(10, 30)
    for i in range(k): 
        #generate one parallel sentence
        
        if stem_sents_rsl[n] is None or stem_sents_rsl[n] == []:
            break
            
        #print("INITIAL GRAM", gram_sents_rus[n])
        #print("INITIAL RUS", stem_sents_rus[n])
        #print("INITIAL RSL", stem_sents_rsl[n])
        new_gram_sent_rus, new_stem_sent_rus, new_stem_rsl = generate_parallel_sentence(gram_sents_rus[n],
                                                                                        stem_sents_rus[n],
                                                                                        stem_sents_rsl[n])
        
        new_gram_sents_rus.append(new_gram_sent_rus)
        new_stem_sents_rus.append(new_stem_sent_rus)
        new_stem_rsls.append(new_stem_rsl)
        #print("NEW GRAM ", new_gram_sent_rus)
        #print("NEW RUS ", new_stem_sent_rus)
        #print("NEW RSL ", new_stem_rsl)
        #print("\n")

100%|██████████| 4876/4876 [00:36<00:00, 132.04it/s]


Save data

In [19]:
#!g1.1
# save to some tsv file
generated_dict = {'new_stem_sent_rus': new_stem_sents_rus, 'new_gram_sent_rus': new_gram_sents_rus, 'new_stem_rsl': new_stem_rsls}
generated_data = pd.DataFrame.from_dict(generated_dict)

In [20]:
#!g1.1
generated_data = generated_data.dropna()

In [21]:
#!g1.1
generated_data.to_csv('generated_stemed.csv', index=False)  

In [22]:
#!g1.1
test_data = pd.DataFrame.from_dict({"test_stem_rus": test_stem_rus, "test_gram_rus": test_gram_rus, "test_rsl": test_rsl})

In [23]:
#!g1.1
test_data = test_data.dropna()

In [24]:
#!g1.1
test_data.to_csv('test_data.csv', index=False)  

Read data

In [3]:
#!g1.1
generated_data = pd.read_csv('generated_stemed.csv', encoding='utf-8')

generated_data['new_stem_sent_rus'] = generated_data['new_stem_sent_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['new_stem_rsl'] = generated_data['new_stem_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['new_gram_sent_rus'] = generated_data['new_gram_sent_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [4]:
#!g1.1
test_data = pd.read_csv('test_data.csv', encoding='utf-8')

test_data['test_stem_rus'] = test_data['test_stem_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['test_rsl'] = test_data['test_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['test_gram_rus'] = test_data['test_gram_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [5]:
#!g1.1
print(len(test_data), len(generated_data))

1220 56740


In [6]:
#!g1.1
len(test_data)/len(generated_data) * 100

2.150158618258724

## Train model

In [7]:
#!g1.1
voc_rus = VocabularyVectorizer(phrase_border=True)
bin_gram = GramBinarizer(phrase_border=True)
voc_rsl = VocabularyVectorizer(phrase_border=True)

In [11]:
#!g1.1
voc_rus = voc_rus.fit(list(generated_data['new_stem_sent_rus']))
bin_gram = bin_gram.fit(list(generated_data['new_gram_sent_rus']))
voc_rsl = voc_rsl.fit(list(generated_data['new_stem_rsl']))

Save vocabs

In [8]:
#!g1.1
# Save\load vocabs
import pickle

def save_vocab(vocab, path):
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()
    
def load_vocab(path):
    output = open(path, 'rb')
    vocab = pickle.load(output)
    output.close()
    return vocab

In [13]:
#!g1.1
save_vocab(voc_rus, 'voc_rus.pkl')
save_vocab(bin_gram, 'bin_gram.pkl')
save_vocab(voc_rsl, 'voc_rsl.pkl')

Load vocabs

In [9]:
#!g1.1
# Load vocabs
voc_rus = load_vocab('voc_rus.pkl')
bin_gram = load_vocab('bin_gram.pkl')
voc_rsl = load_vocab('voc_rsl.pkl')

In [10]:
#!g1.1
voc_rus.index_to_text([[5]])

[['лягушка']]

In [11]:
#!g1.1
vec_sentences_rus_train = voc_rus.text_to_index(list(generated_data['new_stem_sent_rus']))
vec_gram_train = bin_gram.transform(list(generated_data['new_gram_sent_rus']))
vec_sentences_rsl_train = voc_rsl.text_to_index(list(generated_data['new_stem_rsl']))

In [12]:
#!g1.1
words = [w for s in list(generated_data['new_stem_rsl']) for w in s]

Net

In [13]:
#!g1.1
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

from model.model import Translator

In [14]:
#!g1.1
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.
    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better
    
    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
         
        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def normalize_sizes(y_pred, y_true):
    """Normalize tensor sizes
    
    Args:
        y_pred (torch.Tensor): the output of the model
            If a 3-dimensional tensor, reshapes to a matrix
        y_true (torch.Tensor): the target predictions
            If a matrix, reshapes to be a vector
    """
    if len(y_pred.size()) == 3:
        y_pred = y_pred.contiguous().view(-1, y_pred.size(2))
    if len(y_true.size()) == 2:
        y_true = y_true.contiguous().view(-1)
    return y_pred, y_true

def compute_accuracy(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.max(dim=1)
    
    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()
    
    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100

def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

In [15]:
#!g1.1
import random 

def batch_generator(rus_data, rsl_data, batch_size=32):
    rus_data = np.array(rus_data)
    rsl_data = np.array(rsl_data)
    
    data_length = len(rus_data)
    tail_length = batch_size - data_length % batch_size
    index = list(range(data_length))
    random.shuffle(index)
    
    index = np.array(index + random.choices(index, k=tail_length))
    num_batches = len(index) // batch_size
    index = index.reshape((num_batches, batch_size))
    
    for batch_ind, inds in enumerate(tqdm(index)):
        yield batch_ind, torch.tensor(rus_data[inds]), torch.tensor(rsl_data[inds])

In [16]:
#!g1.1
from argparse import Namespace

args = Namespace(
    seed = 1337,
    learning_rate = 5e-4, 
    batch_size = 64,  # 64
    num_epochs = 30,  # 30
    rus_emb_size = 16,  # 16
    rsl_emb_size = 16,  # 16
    rnn_size = 64,  # 64
    early_stopping_criteria = 5,
    mask_index = voc_rsl.mask_ind,
    max_norm = 2.0,
    norm_type = 2
)

set_seed_everywhere(args.seed, torch.cuda.is_available())

model_zero = Translator(voc_rus.word_count, args.rus_emb_size, voc_rsl.word_count, args.rsl_emb_size, args.rnn_size, voc_rsl.bos_ind)

optimizer = optim.Adam(model_zero.parameters(), lr=args.learning_rate)

In [17]:
#!g1.1
print(device)

cuda


In [18]:
#!g1.1
model_zero.to(device)

Translator(
  (encoder): TranslatorEncoder(
    (rus_embeddings): Embedding(14666, 16, max_norm=1.0)
    (rus_birnn): GRU(16, 64, batch_first=True, bidirectional=True)
  )
  (decoder): TranslatorDecoder(
    (rsl_embedding): Embedding(14916, 16, max_norm=1.0)
    (gru_cell): GRUCell(144, 128)
    (linear_map): Linear(in_features=128, out_features=128, bias=True)
    (classifier): Linear(in_features=256, out_features=14916, bias=True)
  )
)

In [19]:
#!g1.1
# train
for epoch_index in range(args.num_epochs):
    # sample_probability = (10 + epoch_index) / args.num_epochs
    if epoch_index < 0.5 * args.num_epochs:
        sample_probability = 0.05
    else:
        sample_probability = ( 2 * (epoch_index+1) - args.num_epochs) / args.num_epochs
    
    running_loss = 0.0
    running_acc = 0.0
    model_zero.train()
    
    for batch_ind, rus_batch, rsl_batch in batch_generator(vec_sentences_rus_train, vec_sentences_rsl_train, args.batch_size):
        optimizer.zero_grad()
        
        rus_batch = rus_batch.to(device)
        rsl_batch = rsl_batch.to(device)
        
        y_pred = model_zero(rus_batch, rsl_batch, sample_probability)  # 0.0
        y_pred = y_pred.to(device)
        
        loss = sequence_loss(y_pred, rsl_batch, args.mask_index)
        
        loss.backward()
        
        #nn.utils.clip_grad_value_(model_zero.parameters(), clip_value=1.0)
        nn.utils.clip_grad_norm_(model_zero.parameters(), args.max_norm, args.norm_type)
        
        optimizer.step()
        
        running_loss += (loss.item() - running_loss) / (batch_ind + 1)
        acc_t = compute_accuracy(y_pred, rsl_batch, args.mask_index)
        
    print('Epoch: ', epoch_index, 'Loss: ', running_loss, '\tacc: ', acc_t, '\tsample_prob: ', sample_probability)

100%|██████████| 887/887 [01:52<00:00,  7.86it/s]


Epoch:  0 Loss:  5.767959586394009 	acc:  43.59903381642512 	sample_prob:  0.05
Epoch:  1 Loss:  2.864862790236745 	acc:  86.68866886688669 	sample_prob:  0.05
Epoch:  2 Loss:  1.4609341984940207 	acc:  84.42796610169492 	sample_prob:  0.05
Epoch:  3 Loss:  0.9825745606637453 	acc:  93.34126040428062 	sample_prob:  0.05
Epoch:  4 Loss:  0.8086890170377978 	acc:  89.05882352941177 	sample_prob:  0.05
Epoch:  5 Loss:  0.7060141920573011 	acc:  85.18134715025907 	sample_prob:  0.05
Epoch:  6 Loss:  0.6171984850537149 	acc:  94.7560975609756 	sample_prob:  0.05
Epoch:  7 Loss:  0.5721194001067991 	acc:  91.46211312700106 	sample_prob:  0.05
Epoch:  8 Loss:  0.5269220709767122 	acc:  95.24348810872027 	sample_prob:  0.05
Epoch:  9 Loss:  0.5216604913285644 	acc:  86.41370869033048 	sample_prob:  0.05
Epoch:  10 Loss:  0.47905228491515595 	acc:  98.28269484808455 	sample_prob:  0.05
Epoch:  11 Loss:  0.4368459423867461 	acc:  88.82618510158014 	sample_prob:  0.05
Epoch:  12 Loss:  0.43913878

In [20]:
#!g1.1
torch.save(model_zero.state_dict(), "model30_gen_clean.pth")

In [41]:
#!g1.1
model_zero = Translator(voc_rus.word_count, args.rus_emb_size,
                        voc_rsl.word_count, args.rsl_emb_size,
                        args.rnn_size, voc_rsl.bos_ind)
model_zero.load_state_dict(torch.load("model30_gen_clean.pth"))

In [21]:
#!g1.1
model_zero.eval()

Translator(
  (encoder): TranslatorEncoder(
    (rus_embeddings): Embedding(14666, 16, max_norm=1.0)
    (rus_birnn): GRU(16, 64, batch_first=True, bidirectional=True)
  )
  (decoder): TranslatorDecoder(
    (rsl_embedding): Embedding(14916, 16, max_norm=1.0)
    (gru_cell): GRUCell(144, 128)
    (linear_map): Linear(in_features=128, out_features=128, bias=True)
    (classifier): Linear(in_features=256, out_features=14916, bias=True)
  )
)

In [22]:
#!g1.1
regex = re.compile(r'(?:<bos>|<eos>.*)')
def pred_vs_rsl(tensor, true_rsl, rus): 
    
    tensor = tensor.argmax(2).tolist()
    
    pred_rsl = voc_rsl.index_to_text(tensor)
    true_rsl = voc_rsl.index_to_text(true_rsl)
    rus = voc_rus.index_to_text(rus)
                   
    f = lambda x: regex.sub('', " ".join(x))
                   
    pred_rsl = [f(sentence) for sentence in pred_rsl]
    true_rsl = [f(sentence) for sentence in true_rsl]
    true_rus = [f(sentence) for sentence in rus]
    
    return pred_rsl, true_rsl, true_rus

In [23]:
#!g1.1
vec_sentences_rus_test = voc_rus.text_to_index(list(test_data['test_stem_rus']))
vec_gram_test = bin_gram.transform(list(test_data['test_gram_rus']))
vec_sentences_rsl_test = voc_rsl.text_to_index(list(test_data['test_rsl']))

In [24]:
#!g1.1
rus = torch.tensor(vec_sentences_rus_test).to(device)
rsl = torch.tensor(vec_sentences_rsl_test).to(device)
y_pred = model_zero(rus,
                    rsl, 1)  #0
trans, truth_rsl, rus = pred_vs_rsl(y_pred,
                                    rsl,
                                    rus)

In [46]:
#!g1.1
voc_rus.word_count, voc_rsl.word_count

(14666, 14916)

In [25]:
#!g1.1
for i in range(10):
    print("RSL: ", truth_rsl[i])
    print("TRANS: ", trans[i])
    print("RUS: ", rus[i])
    print('\n')

RSL:   indx дед.мороз clf <unk> clf.переместить домой clf <unk> clf.переместить дед.мороз маленький <unk> приходить обращение главный 1ps был лес подарок clf <unk> 
TRANS:   все чуть-чуть дед мороз вернуться преподаватель преподаватель дед мороз приходить 1ps 1ps 1ps 1ps 1ps 3ps 3ps 
RUS:   а когда дед мороз вернуться домой маленький дед мороз приходить к главный и сказать а я то вон в лес тоже подарок раздавать 


RSL:   мама второй 3ps все радость давать 1ps ноль 
TRANS:   2ps 1ps утопать 
RUS:   а мачеха лишать я весь радость 


RSL:    
TRANS:   вообще раньше 1ps был indx быть год год год год 1ps у 2ps 1ps indx форум форум форум 1ps indx 1ps indx indx <dact> <dact> indx 1ps 1ps <dact> 1ps 1ps <dact> 1ps 1ps 
RUS:   вообще я думать что мочь быть в следующий год у мы тоже быть план проводить форум и это форум будет включать в себя в то число и вопрос об опыт <unk> в другой регион 


RSL:    
TRANS:   четвертый мама мама 3ps свой дело по 
RUS:   проходить <unk> мама проводить <unk> св

In [26]:
#!g1.1
import sacrebleu

rus_rsl_bleu = sacrebleu.corpus_bleu(trans, [truth_rsl])
print("--------------------------")
print("Russian to RSL: ", rus_rsl_bleu.score)

--------------------------
Russian to RSL:  2.8074663382150127


In [27]:
#!g1.1
# save results to file
result_data = pd.DataFrame.from_dict({"truth RSL": truth_rsl, "TRANS": trans, "RUS": rus})
result_data.to_csv('test_result.csv', index=False)  

### Tests with separate sentences... (Подвал)

In [119]:
#!g1.1
[np.array([0] * 81)]

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [154]:
#!g1.1
[vec_sentences_rus_test[1]]

[array([    2,   351, 10499,  1039,   630,   146,   144,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])]

In [34]:
#!g1.1
y_pred = model_zero(torch.tensor([vec_sentences_rus_test[1]]), torch.tensor([np.array([1] * 81)]), 0)  #torch.tensor(vec_sentences_rsl_test), 0)

In [35]:
#!g1.1
trans, truth_rsl, rus = pred_vs_rsl(y_pred,
                                    torch.tensor([vec_sentences_rsl_test[1]]),
                                    torch.tensor([vec_sentences_rus_test[1]]))

In [36]:
#!g1.1
# lets have a look
for i in range(10):
    print("RSL: ", truth_rsl[i])
    print("TRANS: ", trans[i])
    print("RUS: ", rus[i])
    print('\n')

RSL:   мама второй 3ps все радость давать 1ps ноль 
TRANS:   велосипедист хулиган велосипедист на жопник плато плато стеклянный сам2 на внутри.себя на газированный на clf.стоят на всегда2 газированный внутри.себя на на внутри.себя внутри.себя внутри.себя clf.опуститься clf.опуститься внутри.себя внутри.себя внутри.себя внутри.себя внутри.себя газированный газированный газированный внутри.себя clf.опуститься газированный clf.опуститься clf.опуститься газированный внутри.себя clf.опуститься внутри.себя газированный clf.опуститься внутри.себя газированный clf.опуститься clf.опуститься газированный clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься газированный газированный clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опуститься clf.опустить

IndexError: list index out of range

In [26]:
#!g1.1
# одно предложение с хорошим переводом поменять пару слов местами
vec_sentences_rus_more = voc_rus.text_to_index([['а', 'отец', 'лишать', 'я', 'весь', 'радость']])
vec_sentences_rsl_more = voc_rsl.text_to_index([['мама', 'второй', '3ps', 'все', 'радость', 'давать', '1ps', 'ноль']])
y_pred = model_zero(torch.tensor(vec_sentences_rus_more), torch.tensor(vec_sentences_rsl_more), 1)  #0

In [23]:
#!g1.1
trans, truth_rsl, rus = pred_vs_rsl(y_pred,
                                    torch.tensor(vec_sentences_rsl_more),
                                    torch.tensor(vec_sentences_rus_more))

In [24]:
#!g1.1
print("RSL: ", truth_rsl)
print("TRANS: ", trans)
print("RUS: ", rus)

RSL:  [' мама второй 3ps все радость давать 1ps ноль ']
TRANS:  [' мама второй 3ps все радость давать 1ps ноль ']
RUS:  [' а отец лишать я весь радость ']


In [38]:
#!g1.1
y_pred = model_zero(torch.tensor(vec_sentences_rsl_test), torch.tensor(vec_sentences_rsl_test), 1)  #0
trans, truth_rsl, rus = pred_vs_rsl(y_pred,
                                    torch.tensor(vec_sentences_rsl_test),
                                    torch.tensor(vec_sentences_rus_test))

IndexError: select(): index 14678 out of range for tensor of size [14666, 16] at dimension 0